# **Entendendo os Dados**

### Importação, Leitura e visualização

In [ ]:
#Instalação do Spark

!pip install pyspark

In [ ]:
# Importação das bibliotecas

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.metrics import r2_score

In [ ]:
# Abrindo sessão Spark

spark = SparkSession.builder \
    .appName("Projeto Olist") \
    .getOrCreate()

In [ ]:
# Leitura dos arquivos CSV modificando o separador

orders = spark.read.csv("/content/drive/MyDrive/olist_orders_dataset.csv", header=True, inferSchema=True, sep=",")
products = spark.read.csv("/content/drive/MyDrive/olist_products_dataset.csv", header=True, inferSchema=True, sep=",")
order_items = spark.read.csv("/content/drive/MyDrive/olist_order_items_dataset.csv", header=True, inferSchema=True, sep=",")
payments = spark.read.csv("/content/drive/MyDrive/olist_order_payments_dataset.csv", header=True, inferSchema=True, sep=",")
customers = spark.read.csv("/content/drive/MyDrive/olist_customers_dataset.csv", header=True, inferSchema=True, sep=",")

In [ ]:
# Visualização das tabelas

orders.show(10)
products.show(5)
order_items.show(5)
payments.show(5)
customers.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [ ]:
# Visualização da estrutura

orders.printSchema()
products.printSchema()
order_items.printSchema()
payments.printSchema()
customers.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_

# **Tratamento e Preparação dos Dados**

### Conversão, Criação de Colunas e Joins

In [ ]:
# Verficação de nulos

orders.select([count(when(col(c).isNull(), c)).alias(c) for c in orders.columns]).show()

+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [ ]:
# Conversão do tipo da coluna

orders = orders.withColumn("order_purchase_timestamp", to_timestamp("order_purchase_timestamp", "dd/MM/yyyy HH:mm"))
orders = orders.withColumn("order_approved_at", to_timestamp("order_approved_at", "dd/MM/yyyy HH:mm"))

payments = payments.withColumn("payment_value", col("payment_value").cast("double"))

order_items = order_items.withColumn("shipping_limit_date", to_timestamp("shipping_limit_date", "dd/MM/yyyy HH:mm"))
order_items = order_items.withColumn("price", col("price").cast("double"))
order_items = order_items.withColumn("freight_value", col("freight_value").cast("double"))

In [ ]:
# Criando novas colunas

orders = orders.withColumn("dias_entrega", datediff(col("order_delivered_customer_date"), col("order_purchase_timestamp")))

orders = orders.withColumn("ano", year("order_purchase_timestamp")) \
       .withColumn("mes", month("order_purchase_timestamp"))

In [ ]:
# Join das tabelas

df1 = orders.join(payments, on="order_id", how="inner")
df2 = df1.join(order_items, on="order_id", how="inner")
df_final = df2.join(products, on="product_id", how="left")
df_final2 = df_final.join(customers, on="customer_id", how="left")

# **Análise Exploratória dos Dados**

### Insights com Estatística e Agregação

## **Quantidade Geral**

In [ ]:
# Quantidade de pedidos realizados

orders.select("order_id").distinct().count()

99441

In [ ]:
# Quantidade de pedidos por categoria

df_final.groupBy("product_category_name").count().orderBy("count", ascending=False).show(10)

+---------------------+-----+
|product_category_name|count|
+---------------------+-----+
|      cama_mesa_banho|11823|
|         beleza_saude| 9972|
|        esporte_lazer| 8945|
|     moveis_decoracao| 8744|
| informatica_acess...| 8082|
| utilidades_domest...| 7355|
|   relogios_presentes| 6201|
|            telefonia| 4721|
|   ferramentas_jardim| 4574|
|           automotivo| 4379|
+---------------------+-----+
only showing top 10 rows



In [ ]:
# Quantidade de pedidos por tipo de pagamento

payments.groupBy("payment_type").count().orderBy("count", ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [ ]:
# Clientes recorrentes e únicos

print(f'Clientes recorrentes: {df_final.groupBy("customer_id").count().filter("count > 1").count()}')
print(f'Clientes únicos: {df_final.groupBy("customer_id").count().filter("count = 1").count()}')

Clientes recorrentes: 12463
Clientes únicos: 86202


## **Financeiro**

In [ ]:
# Volume total pago

payments.agg(
    format_number(sum("payment_value"), 2).alias("total_pago")
).show()

+-------------+
|   total_pago|
+-------------+
|16,008,872.12|
+-------------+



In [ ]:
# Total pago por tipo de pagamento

payments.groupBy("payment_type") \
    .agg(format_number(sum("payment_value"), 2).alias("total_pago")) \
    .show()

+------------+-------------+
|payment_type|   total_pago|
+------------+-------------+
|      boleto| 2,869,361.27|
| not_defined|         0.00|
| credit_card|12,542,084.19|
|     voucher|   379,436.87|
|  debit_card|   217,989.79|
+------------+-------------+



In [ ]:
# Total pago por categoria

df_final.groupBy("product_category_name") \
    .agg(format_number(sum("payment_value"), 2).alias("total_pago")) \
    .orderBy("total_pago", ascending=False) \
    .show(10)

+---------------------+----------+
|product_category_name|total_pago|
+---------------------+----------+
|     eletrodomesticos| 95,532.27|
|         climatizacao| 91,170.66|
|            cine_foto|  9,530.32|
|           automotivo|852,294.33|
|        casa_conforto| 84,329.06|
|   ferramentas_jardim|838,280.75|
| fashion_roupa_inf...|    785.67|
|           cool_stuff|779,698.00|
| moveis_cozinha_ar...| 74,660.98|
| construcao_ferram...| 72,712.48|
+---------------------+----------+
only showing top 10 rows



In [ ]:
# Ticket médio do valor por categoria de produto

df_final.groupBy("product_category_name").agg(avg("payment_value").alias("media_categoria")).orderBy("media_categoria", ascending=False).show(10)

+---------------------+------------------+
|product_category_name|   media_categoria|
+---------------------+------------------+
|                  pcs|1268.7343181818185|
|       telefonia_fixa| 763.8754981549814|
| portateis_casa_fo...|  656.786153846154|
| agro_industria_e_...| 471.1532142857142|
|   eletrodomesticos_2|464.78902985074626|
|    moveis_escritorio| 363.7944263217091|
| sinalizacao_e_seg...| 340.7365671641792|
| construcao_ferram...|330.10567839195977|
| instrumentos_musi...|324.61576601671317|
|      eletroportateis| 321.8036804564905|
+---------------------+------------------+
only showing top 10 rows



In [ ]:
# Ticket médio do valor pago por pedido

df_ticket = df_final.select("order_id", "payment_value").dropDuplicates() \
    .groupBy("order_id").agg(sum("payment_value").alias("valor_total"))

df_ticket.agg(avg("valor_total").alias("ticket_medio")).show()

+-----------------+
|     ticket_medio|
+-----------------+
|160.4290819439462|
+-----------------+



In [ ]:
# Desvio padrão do valor pago por pedido

df_final.groupBy("order_id") \
    .agg(sum("payment_value").alias("total")) \
    .agg(stddev("total").alias("desvio_padrao")).show()

# Confirmação do desvio

df_ticket.orderBy("valor_total", ascending=False).show(10, truncate=False)

+-----------------+
|    desvio_padrao|
+-----------------+
|620.7294025421496|
+-----------------+

+--------------------------------+-----------+
|order_id                        |valor_total|
+--------------------------------+-----------+
|03caa2c082116e1d31e67e9ae3700499|13664.08   |
|736e1922ae60d0d6a89247b851902527|7274.88    |
|0812eb902a67711a1cb742b3cdaa65ae|6929.31    |
|fefacc66af859508bf1a7934eab1e97f|6922.21    |
|f5136e38d1a14a4dbd87dff67da82701|6726.66    |
|2cc9089445046817a7539d90805e6e5a|6081.54    |
|a96610ab360d42a2e5335a3998b4718a|4950.34    |
|b4c4b76c642808cbe472a32b86cddc95|4809.44    |
|199af31afc78c699f0dbf71fb178d4d4|4764.34    |
|8dbc85d1447242f3b127dda390d56e19|4681.78    |
+--------------------------------+-----------+
only showing top 10 rows



In [ ]:
# Média de parcelamento por categoria de produto

df_final.groupBy("product_category_name").agg(avg("payment_installments").alias("media_parcelas")).orderBy("media_parcelas", ascending=False).show(10)

+---------------------+------------------+
|product_category_name|    media_parcelas|
+---------------------+------------------+
|                  pcs| 6.009090909090909|
| portateis_casa_fo...| 5.487179487179487|
|           la_cuisine|              4.25|
|   eletrodomesticos_2| 4.104477611940299|
|          moveis_sala| 4.026515151515151|
|        casa_conforto| 3.993684210526316|
|    moveis_escritorio|3.8172103487064115|
| fashion_roupa_fem...|3.7916666666666665|
|   relogios_presentes|3.6729559748427674|
| construcao_ferram...|3.6572029442691902|
+---------------------+------------------+
only showing top 10 rows



## **Tempo e Data**

In [ ]:
# Quantidade de pedidos realizados por ano e por mês

orders.groupBy("ano", "mes") \
  .agg(countDistinct("order_id").alias("total_pedidos")) \
  .orderBy("ano", "mes") \
  .show()

+----+---+-------------+
| ano|mes|total_pedidos|
+----+---+-------------+
|2016|  9|            4|
|2016| 10|          324|
|2016| 12|            1|
|2017|  1|          800|
|2017|  2|         1780|
|2017|  3|         2682|
|2017|  4|         2404|
|2017|  5|         3700|
|2017|  6|         3245|
|2017|  7|         4026|
|2017|  8|         4331|
|2017|  9|         4285|
|2017| 10|         4631|
|2017| 11|         7544|
|2017| 12|         5673|
|2018|  1|         7269|
|2018|  2|         6728|
|2018|  3|         7211|
|2018|  4|         6939|
|2018|  5|         6873|
+----+---+-------------+
only showing top 20 rows



In [ ]:
# Quantidade de pedidos por hora

df_final = df_final.withColumn("hora", hour("order_purchase_timestamp"))
df_final.groupBy("hora").count().orderBy("hora").show(25)

+----+-----+
|hora|count|
+----+-----+
|   0| 2894|
|   1| 1339|
|   2|  611|
|   3|  323|
|   4|  253|
|   5|  225|
|   6|  569|
|   7| 1419|
|   8| 3501|
|   9| 5644|
|  10| 7325|
|  11| 7769|
|  12| 7185|
|  13| 7671|
|  14| 7896|
|  15| 7612|
|  16| 7990|
|  17| 7255|
|  18| 6856|
|  19| 6983|
|  20| 7235|
|  21| 7249|
|  22| 6923|
|  23| 4874|
+----+-----+



In [ ]:
# Quantidade de pedidos por dia da semana

df_final = df_final.withColumn("dia_semana", dayofweek("order_purchase_timestamp"))
df_final.groupBy("dia_semana").count().orderBy("dia_semana").show(10)

+----------+-----+
|dia_semana|count|
+----------+-----+
|         1|13936|
|         2|19130|
|         3|19077|
|         4|18380|
|         5|17590|
|         6|16760|
|         7|12728|
+----------+-----+



In [ ]:
# Tempo médio, mínimo e máximo de entrega

df_final = df_final.withColumn("dias_entrega", datediff(col("order_delivered_customer_date"), col("order_purchase_timestamp")))

df_final.agg(
    avg("dias_entrega").alias("média"),
    min("dias_entrega").alias("mínimo"),
    max("dias_entrega").alias("máximo")
).show()

+------------------+------+------+
|             média|mínimo|máximo|
+------------------+------+------+
|12.422874976094024|     0|   210|
+------------------+------+------+



In [ ]:
# Média de dias de atraso

df_final = df_final.withColumn(
    "atraso_dias",
    datediff("order_delivered_customer_date", "order_estimated_delivery_date")
)

df_final.agg(avg("atraso_dias")).show()

+-------------------+
|   avg(atraso_dias)|
+-------------------+
|-12.043326320913817|
+-------------------+



In [ ]:
# Pedidos entregues com atraso por categoria de produto

df_final.filter("atraso_dias > 0").groupBy("product_category_name") \
    .agg(countDistinct("order_id").alias("num_pedidos")) \
    .orderBy("num_pedidos", ascending=False).show()

+---------------------+-----------+
|product_category_name|num_pedidos|
+---------------------+-----------+
|      cama_mesa_banho|        689|
|         beleza_saude|        649|
|        esporte_lazer|        495|
|     moveis_decoracao|        449|
| informatica_acess...|        417|
|   relogios_presentes|        406|
| utilidades_domest...|        308|
|            telefonia|        291|
|           automotivo|        278|
|           brinquedos|        243|
|                bebes|        226|
|   ferramentas_jardim|        225|
|           cool_stuff|        210|
|           perfumaria|        202|
|          eletronicos|        192|
|            papelaria|        148|
| fashion_bolsas_e_...|        102|
|                 NULL|        101|
|    moveis_escritorio|        101|
|             pet_shop|         86|
+---------------------+-----------+
only showing top 20 rows



In [ ]:
# Pedidos entregues com atraso por mês

df_final.filter("atraso_dias > 0").groupBy("mes") \
  .agg(countDistinct("order_id").alias("total_pedidos")) \
  .orderBy("mes") \
  .show()

+---+-------------+
|mes|total_pedidos|
+---+-------------+
|  1|          425|
|  2|          976|
|  3|         1444|
|  4|          457|
|  5|          549|
|  6|          166|
|  7|          316|
|  8|          515|
|  9|          182|
| 10|          189|
| 11|          904|
| 12|          411|
+---+-------------+



## **Localização**

In [ ]:
# Número de clientes por estado

customers.groupBy("customer_state") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [ ]:
# Número de clientes por cidade (Top 10)

customers.groupBy("customer_city") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(10)

+--------------------+-----+
|       customer_city|count|
+--------------------+-----+
|           sao paulo|15540|
|      rio de janeiro| 6882|
|      belo horizonte| 2773|
|            brasilia| 2131|
|            curitiba| 1521|
|            campinas| 1444|
|        porto alegre| 1379|
|            salvador| 1245|
|           guarulhos| 1189|
|sao bernardo do c...|  938|
+--------------------+-----+
only showing top 10 rows



In [ ]:
# Número de pedidos por estado

df_final2.groupBy("customer_state") \
    .agg(countDistinct("order_id").alias("num_pedidos")) \
    .orderBy("num_pedidos", ascending=False) \
    .show()

+--------------+-----------+
|customer_state|num_pedidos|
+--------------+-----------+
|            SP|      41374|
|            RJ|      12762|
|            MG|      11544|
|            RS|       5432|
|            PR|       4998|
|            SC|       3612|
|            BA|       3358|
|            DF|       2125|
|            ES|       2025|
|            GO|       2007|
|            PE|       1648|
|            CE|       1327|
|            PA|        970|
|            MT|        903|
|            MA|        740|
|            MS|        709|
|            PB|        532|
|            PI|        493|
|            RN|        482|
|            AL|        411|
+--------------+-----------+
only showing top 20 rows



In [ ]:
# Número de clientes por cidade (Top 10)

df_final2.groupBy("customer_city") \
    .agg(countDistinct("order_id").alias("num_pedidos")) \
    .orderBy("num_pedidos", ascending=False) \
    .show(10)

+--------------------+-----------+
|       customer_city|num_pedidos|
+--------------------+-----------+
|           sao paulo|      15402|
|      rio de janeiro|       6834|
|      belo horizonte|       2750|
|            brasilia|       2116|
|            curitiba|       1510|
|            campinas|       1429|
|        porto alegre|       1372|
|            salvador|       1238|
|           guarulhos|       1178|
|sao bernardo do c...|        928|
+--------------------+-----------+
only showing top 10 rows



## **Estatística**

In [ ]:
# Regressão linear para previsão de valor do pedido

df = df_final.toPandas()

X = df[["price", "freight_value"]]
y = df["payment_value"]

modelo = LinearRegression().fit(X, y)
print(f"Coeficientes: {modelo.coef_}")
print(f"Intercepto: {modelo.intercept_}")

y_pred = modelo.predict(X)
print("R²:", r2_score(y, y_pred))

Coeficientes: [1.02424583 1.35439528]
Intercepto: 21.78227648947984
R²: 0.552192481298502


In [ ]:
# Correlação do peso com o preço do frete

df_final.stat.corr("product_weight_g", "freight_value")

0.6118668681435312

# **Exportação**

In [ ]:
df_pedidos_categoria = df_final.groupBy("product_category_name").count().orderBy("count", ascending=False)

df_pedidos_categoria.toPandas().to_csv("/content/pedidos_categoria.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/pedidos_categoria.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_pedidos_pagamento = payments.groupBy("payment_type").count().orderBy("count", ascending=False)

df_pedidos_pagamento.toPandas().to_csv("/content/pedidos_pagamento.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/pedidos_pagamento.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_total_categoria = df_final.groupBy("product_category_name") \
    .agg(format_number(sum("payment_value"), 2).alias("total_pago")) \
    .orderBy("total_pago", ascending=False)

df_total_categoria.toPandas().to_csv("/content/total_categoria.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/total_categoria.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_total_pagamento = payments.groupBy("payment_type") \
    .agg(format_number(sum("payment_value"), 2).alias("total_pago"))

df_total_pagamento.toPandas().to_csv("/content/total_pagamento.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/total_pagamento.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_ticket_categoria = df_final.groupBy("product_category_name").agg(avg("payment_value").alias("media_categoria")).orderBy("media_categoria", ascending=False)

df_ticket_categoria.toPandas().to_csv("/content/ticket_categoria.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/ticket_categoria.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_media_parcelas = df_final.groupBy("product_category_name").agg(avg("payment_installments").alias("media_parcelas")).orderBy("media_parcelas", ascending=False)

df_media_parcelas.toPandas().to_csv("/content/media_parcelas.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/media_parcelas.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_pedidos_tempo = orders.groupBy("ano", "mes") \
  .agg(countDistinct("order_id").alias("total_pedidos")) \
  .orderBy("ano", "mes")

df_pedidos_tempo.toPandas().to_csv("/content/pedidos_tempo.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/pedidos_tempo.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_pedidos_hora = df_final.groupBy("hora").count().orderBy("hora")

df_pedidos_hora.toPandas().to_csv("/content/pedidos_hora.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/pedidos_hora.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_pedidos_dias = df_final.withColumn("dia_semana", dayofweek("order_purchase_timestamp"))
df_pedidos_dias = df_final.groupBy("dia_semana").count().orderBy("dia_semana")

df_pedidos_dias.toPandas().to_csv("/content/pedidos_dias.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/pedidos_dias.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_clientes_estado = customers.groupBy("customer_state") \
    .count() \
    .orderBy("count", ascending=False)

df_clientes_estado.toPandas().to_csv("/content/clientes_estado.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/clientes_estado.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_clientes_cidade = customers.groupBy("customer_city") \
    .count() \
    .orderBy("count", ascending=False)

df_clientes_cidade.toPandas().to_csv("/content/clientes_cidade.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/clientes_cidade.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_total_pago = payments.agg(
    format_number(sum("payment_value"), 2).alias("total_pago")
)

df_total_pago.toPandas().to_csv("/content/total_pago.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/total_pago.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_base_tratada = df_final2.groupBy("order_id").agg(
    first("order_purchase_timestamp").alias("data_pedido"),
    first("customer_state").alias("estado"),
    first("customer_city").alias("cidade"),
    sum("payment_value").alias("valor_pago"),
    sum("freight_value").alias("frete"),
    first("product_category_name").alias("categoria"),
    count("product_id").alias("qtd_itens"),
    first("ano").alias("ano"),
    first("mes").alias("mes"),
)

df_base_tratada.toPandas().to_csv("/content/base_tratada.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/base_tratada.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_atraso_categoria = df_final.filter("atraso_dias > 0").groupBy("product_category_name") \
    .agg(countDistinct("order_id").alias("num_pedidos")) \
    .orderBy("num_pedidos", ascending=False)

df_atraso_categoria.toPandas().to_csv("/content/atraso_categoria.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/atraso_categoria.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_atraso_mes = df_final.filter("atraso_dias > 0").groupBy("mes") \
  .agg(countDistinct("order_id").alias("total_pedidos")) \
  .orderBy("mes")

df_atraso_mes.toPandas().to_csv("/content/atraso_mes.csv", index=False, encoding="utf-8")

from google.colab import files
files.download("/content/atraso_mes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>